In [1]:
import torch
print(torch.cuda.is_available())      # True olmalı
print(torch.cuda.device_count())      # Kaç GPU var?
print(torch.cuda.get_device_name(0)) 

True
1
NVIDIA GeForce GTX 1650


In [2]:
from transformers import AutoTokenizer

prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids
[2132, 572, 264, 6319, 323, 13458, 88]
for t in input_ids:
 print(t, "\t:", tokenizer.decode(t))

c:\Users\egeya\anaconda3\envs\genaibook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


In [3]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

In [4]:
# We tokenize again but specifying the tokenizer that we want it to
# return a PyTorch tensor, which is what the model expects,
# rather than a list of integers
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model(input_ids)
outputs.logits.shape # An output for each input token

torch.Size([1, 7, 151936])

In [6]:
final_logits = model(input_ids).logits[0, -1] # The last set of logits
final_logits.argmax() # The position of the maximum



tensor(3729)

In [7]:
tokenizer.decode(final_logits.argmax())

' night'

In [8]:
import torch
top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
 print(tokenizer.decode(index))

 night
 evening
 day
 morning
 winter
 afternoon
 Saturday
 Sunday
 Friday
 October


In [9]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
 print(f"{tokenizer.decode(index):<10} {value.item():.2%}")


 night     88.71%
 evening   4.30%
 day       2.19%
 morning   0.49%
 winter    0.45%
 afternoon 0.27%
 Saturday  0.25%
 Sunday    0.19%
 Friday    0.17%
 October   0.16%


In [10]:
output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])
print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Input IDs tensor([ 2132,   572,   264,  6319,   323, 13458,    88])
Output IDs tensor([[ 2132,   572,   264,  6319,   323, 13458,    88,  3729,    13,   576,
         12884,   572,  6319,   323,   279,  9956,   572,  1246,  2718,    13,
           576, 11174,   572, 50413,  1495,   323,   279]])
Generated text: It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the


In [11]:
sampling_output = model.generate(
 input_ids,
 do_sample=True,
 max_new_tokens=40,
 top_p=0.94,
 top_k=0,
)
print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


It was a dark and stormy day in my home, and my wife and I were wondering why two burglars went out at night and broke in our home without us noticing.
At first I wondered why they would bother breaking into our
